In [1]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)

from libs.common import *
from config.config import *

c:\Users\Admin\Data\ads_fraud_detection


c:/Users/Admin/Data/ads_fraud_detection


# 3. Tiền xử lí dữ liệu

In [2]:
save_dir=f"{exps_dir}/exp1/exp_gan"

save_dir = f"{save_dir}"
os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

In [3]:
test_size=0.33
seed=42
data=pd.read_excel(f'{exps_dir}/exp1/data_EDA.xlsx')
data

,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,day_policy_bind_date,month_policy_bind_date,year_policy_bind_date,day_incident_date,month_incident_date,year_incident_date
0,Y,Middle,OH,250/500,1000,1406.91,0,MALE,MD,craft-repair,sleeping,husband,53300,0,Single Vehicle Collision,Side Collision,Major Damage,Police,SC,Columbus,Morning,1,YES,1,2,YES,71610,6510,13020,52080,Saab,92x,2002-2008,Y,17,10,2014,25,1,2015
1,Y,Middle,IN,250/500,2000,1197.22,5000000,MALE,MD,machine-op-inspct,reading,other-relative,0,0,Vehicle Theft,?,Minor Damage,Police,VA,Riverwood,Morning,1,?,0,0,?,5070,780,780,3510,Mercedes,E400,2002-2008,Y,27,6,2006,21,1,2015
2,Y,Young,OH,100/300,2000,1413.14,5000000,FEMALE,PhD,sales,board-games,own-child,35100,0,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,NY,Columbus,Morning,3,NO,2,3,NO,34650,7700,3850,23100,Dodge,RAM,2002-2008,N,6,9,2000,22,2,2015
3,Y,Middle,IL,250/500,2000,1415.74,6000000,FEMALE,PhD,armed-forces,board-games,unmarried,48900,-62400,Single Vehicle Collision,Front Collision,Major Damage,Police,OH,Arlington,Morning,1,?,1,2,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2018++,Y,25,5,1990,10,1,2015
4,Y,Middle,IL,500/1000,1000,1583.91,6000000,MALE,Associate,sales,board-games,unmarried,66000,-46000,Vehicle Theft,?,Minor Damage,NaN,NY,Arlington,Night,1,NO,0,1,NO,6500,1300,650,4550,Accura,RSX,2018++,N,6,6,2014,17,2,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,N,Middle,OH,500/1000,1000,1310.80,0,FEMALE,Masters,craft-repair,paintball,unmarried,0,0,Single Vehicle Collision,Front Collision,Minor Damage,Fire,NC,Northbrook,Night,1,YES,0,1,?,87200,17440,8720,61040,Honda,Accord,2002-2008,N,16,7,1991,22,2,2015
996,Y,Middle,IL,100/300,1000,1436.79,0,FEMALE,PhD,prof-specialty,sleeping,wife,70900,0,Single Vehicle Collision,Rear Collision,Major Damage,Fire,SC,Northbend,Night,1,YES,2,3,?,108480,18080,18080,72320,Volkswagen,Passat,2018++,N,5,1,2014,24,1,2015
997,Y,Young,OH,250/500,500,1383.49,3000000,FEMALE,Masters,armed-forces,bungie-jumping,other-relative,35100,0,Multi-vehicle Collision,Side Collision,Minor Damage,Police,NC,Arlington,Morning,3,?,2,3,YES,67500,7500,7500,52500,Suburu,Impreza,1995-20001,N,17,2,2003,23,1,2015
998,Y,Old,IL,500/1000,2000,1356.92,5000000,MALE,Associate,handlers-cleaners,base-jumping,wife,0,0,Single Vehicle Collision,Rear Collision,Major Damage,Other,NY,Arlington,Morning,1,?,0,1,YES,46980,5220,5220,36540,Audi,A5,1995-20001,N,18,11,2011,26,2,2015


In [4]:
data_train, data_test = train_test_split(data, test_size = 0.33, random_state=seed)
print(data_train.shape,data_test.shape)
data_train.to_excel(f'{save_dir}/datatrain.xlsx', index=None)
data_test.to_excel(f'{save_dir}/datatest.xlsx', index=None)

(670, 40) (330, 40)


In [5]:
data_train

,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,day_policy_bind_date,month_policy_bind_date,year_policy_bind_date,day_incident_date,month_incident_date,year_incident_date
703,Y,Middle,IL,250/500,1000,817.28,0,MALE,High School,sales,cross-fit,unmarried,62200,0,Multi-vehicle Collision,Rear Collision,Total Loss,Ambulance,SC,Columbus,Evening,3,?,1,1,NO,60190,4630,9260,46300,Mercedes,ML350,2018++,Y,21,6,1994,3,1,2015
311,N,Young,IN,100/300,2000,765.64,0,MALE,MD,sales,base-jumping,not-in-family,0,-22200,Multi-vehicle Collision,Front Collision,Major Damage,Ambulance,SC,Northbrook,Morning,3,?,0,3,?,31350,2850,5700,22800,Honda,Accord,1995-20001,Y,11,6,2008,16,2,2015
722,Y,Middle,OH,100/300,2000,1422.21,0,FEMALE,Associate,handlers-cleaners,polo,unmarried,0,-46400,Single Vehicle Collision,Side Collision,Major Damage,Ambulance,WV,Riverwood,Night,1,YES,2,1,NO,85900,17180,17180,51540,Suburu,Forrestor,2002-2008,Y,29,7,1996,16,1,2015
629,Y,Middle,IL,100/300,2000,1133.27,0,FEMALE,High School,craft-repair,chess,wife,0,-56400,Multi-vehicle Collision,Side Collision,Total Loss,Other,NY,Northbrook,Night,2,?,0,2,YES,60500,11000,5500,44000,Ford,F150,1995-20001,Y,28,5,2014,1,3,2015
0,Y,Middle,OH,250/500,1000,1406.91,0,MALE,MD,craft-repair,sleeping,husband,53300,0,Single Vehicle Collision,Side Collision,Major Damage,Police,SC,Columbus,Morning,1,YES,1,2,YES,71610,6510,13020,52080,Saab,92x,2002-2008,Y,17,10,2014,25,1,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,Y,Old,OH,500/1000,1000,1125.37,0,FEMALE,Associate,prof-specialty,basketball,husband,0,-56400,Multi-vehicle Collision,Rear Collision,Major Damage,Police,VA,Northbend,Morning,3,?,0,2,YES,79800,6650,19950,53200,Saab,95,1995-20001,Y,7,10,1990,13,1,2015
270,Y,Old,OH,250/500,2000,1589.54,0,MALE,College,handlers-cleaners,camping,husband,55400,0,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,VA,Arlington,Morning,3,?,2,0,YES,85300,17060,8530,59710,Toyota,Highlander,2002-2008,N,15,4,2013,27,1,2015
860,Y,Middle,IN,100/300,1000,1023.11,0,FEMALE,MD,adm-clerical,golf,own-child,0,-45300,Multi-vehicle Collision,Rear Collision,Minor Damage,Other,NY,Northbend,Evening,3,NO,1,2,YES,58850,10700,10700,37450,Accura,MDX,1995-20001,N,16,11,2004,6,2,2015
435,N,Young,OH,250/500,500,1075.41,0,MALE,MD,machine-op-inspct,reading,wife,55200,0,Single Vehicle Collision,Side Collision,Total Loss,Police,PA,Northbend,Morning,1,NO,1,0,NO,73400,7340,7340,58720,Dodge,Neon,1995-20001,N,29,8,1999,15,2,2015


In [6]:
# outliers_list=dict(np.load(f'{save_dir}/outliers.npz'))['outliers']
# def remove_outliers_iqr(data, column):
#     Q1 = data[column].quantile(0.25)
#     Q3 = data[column].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
#     return data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]

# # Áp dụng hàm cho các cột chứa dữ liệu cần loại bỏ ngoại lệ
# for column in outliers_list:
#     data_train = remove_outliers_iqr(data_train, column)
# data_train

In [7]:
data.isnull().sum()

months_as_customer              4
age                             2
policy_state                    0
policy_csl                      0
policy_deductable               0
policy_annual_premium           0
umbrella_limit                  0
insured_sex                     0
insured_education_level         0
insured_occupation              0
insured_hobbies                 0
insured_relationship            0
capital-gains                   0
capital-loss                    0
incident_type                   0
collision_type                  0
incident_severity               0
authorities_contacted          91
incident_state                  0
incident_city                   0
incident_hour_of_the_day       52
number_of_vehicles_involved     0
property_damage                 0
bodily_injuries                 0
witnesses                       0
police_report_available         0
total_claim_amount              0
injury_claim                    0
property_claim                  0
vehicle_claim 

In [8]:
data_train['collision_type']=data_train['collision_type'].replace('?',np.nan)
data_train['collision_type'].fillna(method='ffill',inplace=True)
data_train[['property_damage','police_report_available']]=data_train[['property_damage','police_report_available']].replace('?','Unknow')
(data_train=='?').sum()

months_as_customer             0
age                            0
policy_state                   0
policy_csl                     0
policy_deductable              0
policy_annual_premium          0
umbrella_limit                 0
insured_sex                    0
insured_education_level        0
insured_occupation             0
insured_hobbies                0
insured_relationship           0
capital-gains                  0
capital-loss                   0
incident_type                  0
collision_type                 0
incident_severity              0
authorities_contacted          0
incident_state                 0
incident_city                  0
incident_hour_of_the_day       0
number_of_vehicles_involved    0
property_damage                0
bodily_injuries                0
witnesses                      0
police_report_available        0
total_claim_amount             0
injury_claim                   0
property_claim                 0
vehicle_claim                  0
auto_make 

In [9]:
gen_y_train=data_train['fraud_reported' ]
label_encoder = LabelEncoder()
gen_y_train= label_encoder.fit_transform(gen_y_train)
gen_y_train

array([1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,

In [10]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(data_train['fraud_reported']), y=data_train['fraud_reported'])
class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}
np.savez(f'{save_dir}/class_weights_dict.npz', class_weights_dict = class_weights_dict)
class_weights_dict

{0: 0.6555772994129159, 1: 2.106918238993711}

# 4.Chuẩn hóa dữ liệu

In [11]:
numeric_columns=dict(np.load(f'{exps_dir}/exp1/columns_dtype.npz',allow_pickle=True))['numeric_columns']
numeric_columns

array(['policy_annual_premium', 'capital-gains', 'capital-loss',
       'total_claim_amount', 'injury_claim', 'property_claim',
       'vehicle_claim', 'day_policy_bind_date', 'month_policy_bind_date',
       'year_policy_bind_date', 'day_incident_date',
       'month_incident_date', 'year_incident_date'], dtype='<U22')

In [12]:
category_columns=dict(np.load(f'{exps_dir}/exp1/columns_dtype.npz',allow_pickle=True))['category_columns']
category_columns

array(['months_as_customer', 'age', 'policy_state', 'policy_csl',
       'policy_deductable', 'umbrella_limit', 'insured_sex',
       'insured_education_level', 'insured_occupation', 'insured_hobbies',
       'insured_relationship', 'incident_type', 'collision_type',
       'incident_severity', 'authorities_contacted', 'incident_state',
       'incident_city', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'property_damage',
       'bodily_injuries', 'witnesses', 'police_report_available',
       'auto_make', 'auto_model', 'auto_year', 'fraud_reported'],
      dtype='<U27')

In [13]:
data_train

,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,day_policy_bind_date,month_policy_bind_date,year_policy_bind_date,day_incident_date,month_incident_date,year_incident_date
703,Y,Middle,IL,250/500,1000,817.28,0,MALE,High School,sales,cross-fit,unmarried,62200,0,Multi-vehicle Collision,Rear Collision,Total Loss,Ambulance,SC,Columbus,Evening,3,Unknow,1,1,NO,60190,4630,9260,46300,Mercedes,ML350,2018++,Y,21,6,1994,3,1,2015
311,N,Young,IN,100/300,2000,765.64,0,MALE,MD,sales,base-jumping,not-in-family,0,-22200,Multi-vehicle Collision,Front Collision,Major Damage,Ambulance,SC,Northbrook,Morning,3,Unknow,0,3,Unknow,31350,2850,5700,22800,Honda,Accord,1995-20001,Y,11,6,2008,16,2,2015
722,Y,Middle,OH,100/300,2000,1422.21,0,FEMALE,Associate,handlers-cleaners,polo,unmarried,0,-46400,Single Vehicle Collision,Side Collision,Major Damage,Ambulance,WV,Riverwood,Night,1,YES,2,1,NO,85900,17180,17180,51540,Suburu,Forrestor,2002-2008,Y,29,7,1996,16,1,2015
629,Y,Middle,IL,100/300,2000,1133.27,0,FEMALE,High School,craft-repair,chess,wife,0,-56400,Multi-vehicle Collision,Side Collision,Total Loss,Other,NY,Northbrook,Night,2,Unknow,0,2,YES,60500,11000,5500,44000,Ford,F150,1995-20001,Y,28,5,2014,1,3,2015
0,Y,Middle,OH,250/500,1000,1406.91,0,MALE,MD,craft-repair,sleeping,husband,53300,0,Single Vehicle Collision,Side Collision,Major Damage,Police,SC,Columbus,Morning,1,YES,1,2,YES,71610,6510,13020,52080,Saab,92x,2002-2008,Y,17,10,2014,25,1,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,Y,Old,OH,500/1000,1000,1125.37,0,FEMALE,Associate,prof-specialty,basketball,husband,0,-56400,Multi-vehicle Collision,Rear Collision,Major Damage,Police,VA,Northbend,Morning,3,Unknow,0,2,YES,79800,6650,19950,53200,Saab,95,1995-20001,Y,7,10,1990,13,1,2015
270,Y,Old,OH,250/500,2000,1589.54,0,MALE,College,handlers-cleaners,camping,husband,55400,0,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,VA,Arlington,Morning,3,Unknow,2,0,YES,85300,17060,8530,59710,Toyota,Highlander,2002-2008,N,15,4,2013,27,1,2015
860,Y,Middle,IN,100/300,1000,1023.11,0,FEMALE,MD,adm-clerical,golf,own-child,0,-45300,Multi-vehicle Collision,Rear Collision,Minor Damage,Other,NY,Northbend,Evening,3,NO,1,2,YES,58850,10700,10700,37450,Accura,MDX,1995-20001,N,16,11,2004,6,2,2015
435,N,Young,OH,250/500,500,1075.41,0,MALE,MD,machine-op-inspct,reading,wife,55200,0,Single Vehicle Collision,Side Collision,Total Loss,Police,PA,Northbend,Morning,1,NO,1,0,NO,73400,7340,7340,58720,Dodge,Neon,1995-20001,N,29,8,1999,15,2,2015


In [14]:
# import pandas as pd
# from sklearn.preprocessing import OneHotEncoder

# # Giả sử bạn đã có một DataFrame tên là data_train và các cột phân loại
# data_train = pd.DataFrame({
#     'loai_xe': ['SUV', 'sedan', 'SUV', 'hatchback'],
#     'mau_sac': ['red', 'blue', 'green', 'blue'],
#     'gia': [20000, 15000, 22000, 18000]
# })

# # Các cột phân loại
# category_columns = ['loai_xe', 'mau_sac']

# label_encoders = {}

# for column in category_columns:
#     encoder = OneHotEncoder(sparse_output=False)
    
#     # Chuyển đổi cột thành 2D và thực hiện One-Hot Encoding
#     encoded = encoder.fit_transform(data_train[[column]])  # Nhớ rằng phải truyền DataFrame 2D
    
#     # Chuyển kết quả One-Hot Encoding thành DataFrame với cùng chỉ số
#     encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out([column]), index=data_train.index)
    
#     # Bỏ cột gốc và thêm các cột đã được One-Hot Encoding vào data_train
#     data_train = pd.concat([data_train.drop(column, axis=1), encoded_df], axis=1)
    
#     # Lưu encoder để có thể sử dụng lại sau này
#     label_encoders[column] = encoder

# print(data_train)


In [15]:
data_train['collision_type']=data_train['collision_type'].replace('?',np.nan)
data_train['collision_type'].fillna(method='ffill',inplace=True)
data_train[['property_damage','police_report_available']]=data_train[['property_damage','police_report_available']].replace('?','Unknow')
(data_train=='?').sum()

months_as_customer             0
age                            0
policy_state                   0
policy_csl                     0
policy_deductable              0
policy_annual_premium          0
umbrella_limit                 0
insured_sex                    0
insured_education_level        0
insured_occupation             0
insured_hobbies                0
insured_relationship           0
capital-gains                  0
capital-loss                   0
incident_type                  0
collision_type                 0
incident_severity              0
authorities_contacted          0
incident_state                 0
incident_city                  0
incident_hour_of_the_day       0
number_of_vehicles_involved    0
property_damage                0
bodily_injuries                0
witnesses                      0
police_report_available        0
total_claim_amount             0
injury_claim                   0
property_claim                 0
vehicle_claim                  0
auto_make 

In [16]:
label_encoders = {}
for column in category_columns:
    encoder = OneHotEncoder(sparse_output=False)
    
    encoded = encoder.fit_transform(data_train[[column]])  
    
    encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out([column]), index=data_train.index)
    
    data_train = pd.concat([data_train.drop(column, axis=1), encoded_df], axis=1)
    
    label_encoders[column] = encoder


scaler = StandardScaler()
data_train[numeric_columns] = scaler.fit_transform(data_train[numeric_columns])
data_train

,policy_annual_premium,capital-gains,capital-loss,total_claim_amount,injury_claim,property_claim,vehicle_claim,day_policy_bind_date,month_policy_bind_date,year_policy_bind_date,day_incident_date,month_incident_date,year_incident_date,months_as_customer_N,months_as_customer_Y,months_as_customer_nan,age_Middle,age_Old,age_Young,age_nan,policy_state_IL,policy_state_IN,policy_state_OH,policy_csl_100/300,policy_csl_250/500,policy_csl_500/1000,policy_deductable_500,policy_deductable_1000,policy_deductable_2000,umbrella_limit_0,umbrella_limit_2000000,umbrella_limit_3000000,umbrella_limit_4000000,umbrella_limit_5000000,umbrella_limit_6000000,umbrella_limit_7000000,umbrella_limit_8000000,umbrella_limit_9000000,umbrella_limit_10000000,insured_sex_FEMALE,insured_sex_MALE,insured_education_level_Associate,insured_education_level_College,insured_education_level_High School,insured_education_level_JD,insured_education_level_MD,insured_education_level_Masters,insured_education_level_PhD,insured_occupation_adm-clerical,insured_occupation_armed-forces,insured_occupation_craft-repair,insured_occupation_exec-managerial,insured_occupation_farming-fishing,insured_occupation_handlers-cleaners,insured_occupation_machine-op-inspct,insured_occupation_other-service,insured_occupation_priv-house-serv,insured_occupation_prof-specialty,insured_occupation_protective-serv,insured_occupation_sales,insured_occupation_tech-support,insured_occupation_transport-moving,insured_hobbies_base-jumping,insured_hobbies_basketball,insured_hobbies_board-games,insured_hobbies_bungie-jumping,insured_hobbies_camping,insured_hobbies_chess,insured_hobbies_cross-fit,insured_hobbies_dancing,insured_hobbies_exercise,insured_hobbies_golf,insured_hobbies_hiking,insured_hobbies_kayaking,insured_hobbies_movies,insured_hobbies_paintball,insured_hobbies_polo,insured_hobbies_reading,insured_hobbies_skydiving,insured_hobbies_sleeping,insured_hobbies_video-games,insured_hobbies_yachting,insured_relationship_husband,insured_relationship_not-in-family,insured_relationship_other-relative,insured_relationship_own-child,insured_relationship_unmarried,insured_relationship_wife,incident_type_Multi-vehicle Collision,incident_type_Parked Car,incident_type_Single Vehicle Collision,incident_type_Vehicle Theft,collision_type_Front Collision,collision_type_Rear Collision,collision_type_Side Collision,incident_severity_Major Damage,incident_severity_Minor Damage,incident_severity_Total Loss,incident_severity_Trivial Damage,authorities_contacted_Ambulance,authorities_contacted_Fire,authorities_contacted_Other,authorities_contacted_Police,authorities_contacted_nan,incident_state_NC,incident_state_NY,incident_state_OH,incident_state_PA,incident_state_SC,incident_state_VA,incident_state_WV,incident_city_Arlington,incident_city_Columbus,incident_city_Hillsdale,incident_city_Northbend,incident_city_Northbrook,incident_city_Riverwood,incident_city_Springfield,incident_hour_of_the_day_Evening,incident_hour_of_the_day_Morning,incident_hour_of_the_day_Night,incident_hour_of_the_day_nan,number_of_vehicles_involved_1,number_of_vehicles_involved_2,number_of_vehicles_involved_3,number_of_vehicles_involved_4,property_damage_NO,property_damage_Unknow,property_damage_YES,bodily_injuries_0,bodily_injuries_1,bodily_injuries_2,witnesses_0,witnesses_1,witnesses_2,witnesses_3,police_report_available_NO,police_report_available_Unknow,police_report_available_YES,auto_make_Accura,auto_make_Audi,auto_make_BMW,auto_make_Chevrolet,auto_make_Dodge,auto_make_Ford,auto_make_Honda,auto_make_Jeep,auto_make_Mercedes,auto_make_Nissan,auto_make_Saab,auto_make_Suburu,auto_make_Toyota,auto_make_Volkswagen,auto_model_3 Series,auto_model_92x,auto_model_93,auto_model_95,auto_model_A3,auto_model_A5,auto_model_Accord,auto_model_C300,auto_model_CRV,auto_model_Camry,auto_model_Civic,auto_model_Corolla,auto_model_E400,auto_model_Escape,auto_model_F150,auto_model_Forrestor,auto_model_Fusion,auto_model_Grand Cherokee,auto_model_Highlander,auto_mode

In [17]:
data_train.isnull().sum()

policy_annual_premium                     0
capital-gains                             0
capital-loss                              0
total_claim_amount                        0
injury_claim                              0
property_claim                            0
vehicle_claim                             0
day_policy_bind_date                      0
month_policy_bind_date                    0
year_policy_bind_date                     0
day_incident_date                         0
month_incident_date                       0
year_incident_date                        0
months_as_customer_N                      0
months_as_customer_Y                      0
months_as_customer_nan                    0
age_Middle                                0
age_Old                                   0
age_Young                                 0
age_nan                                   0
policy_state_IL                           0
policy_state_IN                           0
policy_state_OH                 

In [18]:
x_train=data_train.drop(columns=['fraud_reported_N' ,'fraud_reported_Y'],axis=1)
y_train=data_train[['fraud_reported_N' ,'fraud_reported_Y']]

# x_test=data_test.drop('fraud_reported',axis=1)
# y_test=data_test['fraud_reported']

In [19]:
y_train.value_counts()

fraud_reported_N  fraud_reported_Y
1.0               0.0                 511
0.0               1.0                 159
Name: count, dtype: int64

In [20]:
class_0_count = np.sum(y_train == 0)
class_1_count = np.sum(y_train == 1)
gen_x_times = class_0_count / class_1_count
print(f"Lớp 0 (đa số): {class_0_count} mẫu")
print(f"Lớp 1 (thiểu số): {class_1_count} mẫu")

Lớp 0 (đa số): fraud_reported_N    159
fraud_reported_Y    511
dtype: int64 mẫu
Lớp 1 (thiểu số): fraud_reported_N    511
fraud_reported_Y    159
dtype: int64 mẫu


In [21]:
x_train_minority = x_train[y_train == 1]
y_train_minority = y_train[y_train == 1]

In [22]:
print(gen_x_times)

fraud_reported_N    0.311155
fraud_reported_Y    3.213836
dtype: float64


In [23]:
print(y_train)

     fraud_reported_N  fraud_reported_Y
703               0.0               1.0
311               0.0               1.0
722               0.0               1.0
629               0.0               1.0
0                 0.0               1.0
..                ...               ...
106               0.0               1.0
270               1.0               0.0
860               1.0               0.0
435               1.0               0.0
102               1.0               0.0

[670 rows x 2 columns]


In [24]:
# gen_y_train = gen_y_train.to_frame()
# # x_train_minority= x_train_minority.to_frame()
# y_train_minority = y_train_minority.to_frame()


In [25]:
# print(type(x_train_balanced))  # Phải là <class 'pandas.core.frame.DataFrame'>
# print(type(y_train_balanced))  # Phải là <class 'pandas.core.frame.DataFrame'>


In [26]:
object_cols_test=dict(np.load(f'{exps_dir}/exp1/columns_dtype.npz',allow_pickle=True))['category_columns']
numeric_cols_test=dict(np.load(f'{exps_dir}/exp1/columns_dtype.npz',allow_pickle=True))['numeric_columns']
# for column in outliers_list:
#     data_test = remove_outliers_iqr(data_test, column)
# data_test['collision_type'].replace('?',np.nan,inplace=True)
# data_test['collision_type'].fillna(method='ffill',inplace=True)
# data_test[['property_damage','police_report_available']].replace('?','Unknow',inplace=True)
# (data_test=='?').sum()
label_encoders = {}
gen_y_test=data_test['fraud_reported' ]
label_encoder = LabelEncoder()
gen_y_test= label_encoder.fit_transform(gen_y_test)
for column in category_columns:
    encoder = OneHotEncoder(sparse_output=False)
    
    encoded = encoder.fit_transform(data_test[[column]])  
    
    encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out([column]), index=data_test.index)
    
    data_test = pd.concat([data_test.drop(column, axis=1), encoded_df], axis=1)
    
    label_encoders[column] = encoder

scaler = StandardScaler()
data_test[numeric_cols_test] = scaler.fit_transform(data_test[numeric_cols_test])

x_test=data_test.drop(columns=['fraud_reported_N' ,'fraud_reported_Y'],axis=1)
y_test=data_test[['fraud_reported_N' ,'fraud_reported_Y']]
data_test.to_excel(f'{save_dir}/df_test_minmax.xlsx', index=False)
x_test.to_excel(f'{save_dir}/x_test.xlsx', index=False)
y_test.to_excel(f'{save_dir}/y_test.xlsx', index=False)

In [27]:
x_test_minority = x_test[y_test == 1]

In [28]:
y_train_minority.value_counts()

Series([], Name: count, dtype: int64)

In [29]:
gen_y_train

array([1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,

In [30]:
gen_y_test

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0,

In [31]:
import pandas as pd

# Kiểm tra xem có giá trị NaN trong DataFrame không
print(x_train.isnull().sum())  # Kiểm tra tổng số giá trị NaN trong mỗi cột


policy_annual_premium                     0
capital-gains                             0
capital-loss                              0
total_claim_amount                        0
injury_claim                              0
property_claim                            0
vehicle_claim                             0
day_policy_bind_date                      0
month_policy_bind_date                    0
year_policy_bind_date                     0
day_incident_date                         0
month_incident_date                       0
year_incident_date                        0
months_as_customer_N                      0
months_as_customer_Y                      0
months_as_customer_nan                    0
age_Middle                                0
age_Old                                   0
age_Young                                 0
age_nan                                   0
policy_state_IL                           0
policy_state_IN                           0
policy_state_OH                 

In [32]:
print(gen_y_train.shape)

(670,)


In [33]:
# from imblearn.over_sampling import SMOTE
# x_train, y_train = SMOTE().fit_resample(x_train, y_train)
from tabgan.sampler import GANGenerator

gen_x, gen_y = GANGenerator(gen_x_times=10.5, 
                                cat_cols=None,
                                bot_filter_quantile=0.001, 
                                top_filter_quantile=0.999,
                                is_post_process=True,
                                adversarial_model_params={
                                    "metrics": "accuracy", 
                                    "max_depth": 2, 
                                    "max_bin": 100, 
                                    "learning_rate": 0.02, 
                                    "random_state": 42, 
                                    "n_estimators": 500,
                                },
                                pregeneration_frac=2,
                                only_generated_data=False,
                                gen_params = {
                                    "batch_size": 500, 
                                    "patience": 25, 
                                    "epochs" : 500,
                                }).generate_data_pipe(x_train, pd.DataFrame(gen_y_train), x_test, 
                                                      deep_copy=True, only_adversarial=False, 
                                                      use_adversarial=True)

print(gen_x.shape, gen_y.shape)

Fitting CTGAN transformers for each column:   0%|          | 0/196 [00:00<?, ?it/s]

ValueError: Input X contains NaN.
BayesianGaussianMixture does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
gen_x

In [431]:
x_test.isnull().sum()

policy_annual_premium                     0
capital-gains                             0
capital-loss                              0
total_claim_amount                        0
injury_claim                              0
property_claim                            0
vehicle_claim                             0
day_policy_bind_date                      0
month_policy_bind_date                    0
year_policy_bind_date                     0
day_incident_date                         0
month_incident_date                       0
year_incident_date                        0
months_as_customer_N                      0
months_as_customer_Y                      0
months_as_customer_nan                    0
age_Middle                                0
age_Old                                   0
age_Young                                 0
age_nan                                   0
policy_state_IL                           0
policy_state_IN                           0
policy_state_OH                 

In [25]:
data_train.shape

(670, 40)

In [37]:
print(gen_x.shape, gen_y.shape)

(25, 39) (25,)


In [36]:
gen_y.value_counts()

fraud_reported
0    14
1    11
Name: count, dtype: int64

In [28]:
gen_x.to_excel(f'{save_dir}/gen_x.xlsx', index=False)
gen_y.to_excel(f'{save_dir}/gen_y.xlsx', index=False)

In [29]:
y_train

,fraud_reported
703,1
311,1
722,1
629,1
0,1
...,...
106,1
270,0
860,0
435,0


In [30]:
x_train

,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,day_policy_bind_date,month_policy_bind_date,year_policy_bind_date,day_incident_date,month_incident_date,year_incident_date
703,1,0,0,1,1,-1.770734,0,1,2,11,6,4,1.270697,0.970481,0,2,2,0,4,1,0,2,0,1,1,1,0.249230,-0.612776,0.358110,0.416329,8,24,2,0.579574,-0.179439,-1.020460,-1.393943,-0.947412,0.0
311,0,2,1,0,2,-1.979562,0,1,4,11,0,1,-0.919382,0.175400,0,1,0,0,4,4,1,2,0,0,3,0,-0.845210,-0.976821,-0.374783,-0.834115,6,6,0,-0.547936,-0.179439,0.908584,0.115223,0.953085,0.0
722,1,0,2,0,2,0.675544,0,0,0,5,14,4,-0.919382,-0.691310,2,3,0,0,6,5,2,0,2,2,1,1,1.224890,1.953951,1.988592,0.695152,11,15,1,1.481582,0.108179,-0.744883,0.115223,-0.947412,0.0
629,1,0,0,0,2,-0.492901,0,0,2,2,5,5,-0.919382,-1.049454,0,3,2,2,1,4,2,1,0,0,2,2,0.260994,0.690017,-0.415957,0.293945,5,14,0,1.368831,-0.467057,1.735317,-1.626123,2.853582,0.0
0,1,0,2,1,1,0.613672,0,1,4,2,17,0,0.957325,0.970481,2,3,0,3,4,1,1,0,2,1,2,2,0.682604,-0.228278,1.132177,0.723886,10,1,1,0.128570,0.971032,1.735317,1.160031,-0.947412,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,1,1,2,2,1,-0.524848,0,0,0,9,1,0,-0.919382,-1.049454,0,2,0,3,5,3,1,2,0,0,2,2,0.993403,-0.199645,2.558849,0.783481,10,3,0,-0.998940,0.971032,-1.571616,-0.233046,-0.947412,0.0
270,1,1,2,1,2,1.352210,0,1,1,5,4,0,1.031267,0.970481,0,2,1,3,5,0,1,2,0,2,0,2,1.202121,1.929408,0.207826,1.129881,12,18,1,-0.096932,-0.754675,1.597528,1.392211,-0.947412,0.0
860,1,0,1,0,1,-0.938378,0,0,4,0,9,3,-0.919382,-0.651914,0,2,1,2,1,3,0,2,1,1,2,2,0.198379,0.628661,0.654562,-0.054583,0,23,0,0.015819,1.258650,0.357428,-1.045674,0.953085,0.0
435,0,2,2,1,0,-0.726882,0,1,4,6,15,5,1.024225,0.970481,2,3,2,3,3,3,1,0,1,1,0,1,0.750532,-0.058526,-0.037158,1.077203,4,27,0,1.481582,0.395797,-0.331516,-0.000866,0.953085,0.0


In [31]:
gen_x

,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,day_policy_bind_date,month_policy_bind_date,year_policy_bind_date,day_incident_date,month_incident_date,year_incident_date
0,0,2,1,1,1,0.079028,4,1,1,9,10,2,0.753106,-1.131827,0,2,0,0,0,6,0,2,2,2,3,2,0.067456,-0.426662,0.732792,0.016187,3,32,1,0.917827,1.546268,0.633006,-0.465225,-0.947412,0.0
1,1,0,0,1,2,1.922199,6,1,1,13,10,1,0.710853,-0.522982,0,1,1,1,6,4,0,2,1,2,0,1,0.096297,-0.602550,0.378697,0.193910,10,3,0,1.368831,-0.754675,1.597528,1.043941,-0.947412,0.0
2,1,2,0,0,1,0.828201,5,0,2,0,15,5,0.534802,-0.533726,0,3,2,0,1,3,0,2,1,0,1,1,-0.452821,-0.009441,0.012251,-0.635641,10,3,0,0.917827,-1.329911,1.459740,-0.929584,-0.947412,0.0
3,1,0,0,1,1,0.176688,5,0,2,11,4,2,1.591110,-1.547275,0,1,0,0,1,1,0,2,0,1,3,2,0.031405,-0.322357,0.942778,-0.115775,10,1,1,-0.998940,1.546268,-0.469305,-1.277853,-0.947412,0.0
4,1,2,1,2,0,0.010807,6,1,2,4,15,3,0.513676,-0.519400,0,2,0,2,2,0,1,3,1,2,2,1,-0.224749,-0.584143,0.415754,-0.270617,0,33,2,-1.562696,0.395797,-1.571616,-0.116956,0.953085,0.0
5,0,2,2,0,1,-0.817344,6,0,5,7,15,3,0.911552,-0.623262,0,2,1,0,0,4,2,2,1,1,3,0,0.023435,0.289158,0.312819,-0.123224,10,2,2,0.805076,-0.179439,1.459740,1.043941,0.953085,0.0
6,0,2,2,0,0,0.596971,4,0,6,7,0,0,1.555900,-0.830986,2,2,1,0,1,4,1,0,2,0,1,0,0.092881,-0.285544,1.016891,-0.058308,12,9,2,-0.998940,-1.042293,0.908584,-1.510033,0.953085,0.0
7,1,1,2,2,0,-0.771041,4,1,0,8,5,2,1.668573,-0.580285,2,1,2,0,6,0,2,0,1,2,1,0,0.351312,-0.488018,0.609270,0.462090,8,12,1,0.579574,1.258650,0.633006,-0.233046,0.953085,0.0
8,1,0,2,0,1,-0.976795,5,0,0,2,12,3,0.689727,-0.512237,1,0,1,4,1,6,1,0,0,1,1,2,-1.822768,-1.383816,-1.371188,-1.841389,11,19,1,0.015819,0.683414,1.597528,-1.393943,-0.947412,0.0
9,1,2,2,0,1,0.319438,5,1,2,2,15,1,0.957325,-0.791590,2,1,0,2,6,6,0,0,2,2,0,2,-0.019826,0.612299,-0.455072,-0.069482,8,7,0,-0.886189,0.395797,1.184162,1.043941,-0.947412,0.0


In [32]:
gen_y

0     1
1     0
2     0
3     0
4     1
5     1
6     0
7     1
8     0
9     1
10    0
11    1
12    0
13    1
14    1
15    0
16    1
17    1
18    0
19    0
20    0
21    0
22    0
23    1
24    0
Name: fraud_reported, dtype: int32

In [33]:
print(data_train.shape)

(670, 40)


In [32]:
x_train.to_excel(f'{save_dir}/x_train.xlsx', index=False)
y_train.to_excel(f'{save_dir}/y_train.xlsx', index=False)

In [33]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}
np.savez(f'{save_dir}/class_weights_dict.npz', class_weights_dict = class_weights_dict)
class_weights_dict


{0: 1.0352032938754503, 1: 0.9671122223290701}

In [34]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
print(f"+ X_train: {len(x_train)}")
for fold, (train_idx, valid_idx) in enumerate(kfold.split(x_train, y_train)):
    print(f'Fold {fold}: ')
    print(f'+ train_idx: {train_idx}')
    print(f'+ valid_idx: {valid_idx}')
    print(f'+ train / valid: {valid_idx}')
    pass

+ X_train: 20114
Fold 0: 
+ train_idx: [    1     2     4 ... 20110 20111 20113]
+ valid_idx: [    0     3     6 ... 20105 20109 20112]
+ train / valid: [    0     3     6 ... 20105 20109 20112]
Fold 1: 
+ train_idx: [    0     1     2 ... 20111 20112 20113]
+ valid_idx: [    5     8    14 ... 20102 20107 20110]
+ train / valid: [    5     8    14 ... 20102 20107 20110]
Fold 2: 
+ train_idx: [    0     1     3 ... 20111 20112 20113]
+ valid_idx: [    2     7    10 ... 20089 20103 20104]
+ train / valid: [    2     7    10 ... 20089 20103 20104]
Fold 3: 
+ train_idx: [    0     2     3 ... 20111 20112 20113]
+ valid_idx: [    1     4    18 ... 20090 20098 20108]
+ train / valid: [    1     4    18 ... 20090 20098 20108]
Fold 4: 
+ train_idx: [    0     1     2 ... 20109 20110 20112]
+ valid_idx: [    9    11    13 ... 20106 20111 20113]
+ train / valid: [    9    11    13 ... 20106 20111 20113]
